In [ ]:
import pandas as pd

import datareader
import problemsolver
import solutions

In [ ]:
FILE_PREFERENCES = "Voorkeuren en wensen groepsindeling - Copy_zonder_2.xlsx"
FILE_GROUPS_TO = "groepen_2025.xlsx"

In [ ]:
groups_to = pd.read_excel(FILE_GROUPS_TO, index_col=0).to_dict(orient="index")
processor = datareader.VoorkeurenProcessor(FILE_PREFERENCES)
preferences = processor.process(all_to_groups=list(groups_to.keys()))
students_info = processor.get_students_meta_info()

In [ ]:
df_groups_to = pd.DataFrame.from_dict(groups_to, orient="index")
display(df_groups_to.assign(Totaal=lambda df: df.sum("columns")))


df_students = pd.DataFrame.from_dict(students_info, orient="index")
print(df_students[["Jongen/meisje"]].value_counts())
print(df_students["Stamgroep"].value_counts())

In [ ]:
prob_name = "relative_satisfaction"
filename = f"{prob_name}.json"
ps = problemsolver.ProblemSolver(
    preferences,
    students_info,
    groups_to,
    max_imbalance_boys_girls_total=6,
    optimize="least_satisfied",
)


ps.run()
ps.save(filename)

In [ ]:
sa = solutions.SolutionAnalyzer(filename, preferences, processor.input, students_info)

display(sa.display_groepsindeling())
display(sa.solution_performance)
display(sa.display_transition_matrix())
display(sa.group_report)
display(sa.display_student_performance())
display(sa.display_satisfied_preferences())
sa.to_excel()

# Analysis

In [ ]:
solution_performance_overview = dict()
known_hashes = set()
relevant_solutions = []
optimization_options = [
    "studentsatisfaction",
    "weighted_preferences",
    "least_satisfied",
]
for optimize in optimization_options:
    for max_imbalance_boys_girls_year in range(1, 4):
        ps = problemsolver.ProblemSolver(
            preferences,
            students_info,
            groups_to,
            optimize=optimize,
            max_imbalance_boys_girls_year=max_imbalance_boys_girls_year,
        )
        prob = ps.run()
        sa = solutions.SolutionAnalyzer(prob, preferences, processor.input)
        new_hash = sa.get_hash()
        if new_hash not in known_hashes:
            known_hashes.add(new_hash)
            relevant_solutions.append(sa)
            print(f"{optimize=} {max_imbalance_boys_girls_year=}")
            display(sa.display_groepsindeling())

            solution_performance_overview[(optimize, max_imbalance_boys_girls_year)] = (
                sa.solution_performance
            )

In [ ]:
pd.DataFrame.from_dict(solution_performance_overview, orient="index")

In [ ]:
for sol in relevant_solutions:
    display(sol.display_groepsindeling())
    display(sol.solution_performance)
    display(sol.display_student_performance())
    print("-" * 30)